In [1]:
import numpy as np
import pandas as pd
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier

In [2]:
df=pd.read_excel("Attended_College.xlsx")
df['Atteded_College'].replace({"Y":1,"N":-1},inplace=True)

In [3]:
df

,Age,Salary,Atteded_College
0,25,4500,1
1,29,4700,-1
2,32,12300,1
3,24,6700,1
4,26,5600,-1
5,25,2900,-1
6,30,11700,-1
7,31,14500,-1
8,27,6500,1
9,28,7800,-1


In [4]:
X=df[["Age","Salary"]]
y=df["Atteded_College"]
df['weights']=1/9
model=DecisionTreeClassifier(max_depth=1)
fit=model.fit(X,y)
pre_y=model.predict(X)


df["predicted_y"]=pre_y
df["y*yhat"]=df.Atteded_College*df.predicted_y
accuracy = np.sum(y == pre_y)/len(y)
error = 1 - accuracy
alpha = 0.5*np.log((1 - error)/error)
df['new_weight']=df.weights*np.exp(-1*df['y*yhat']*alpha)
df['normalized_weight']=df['new_weight']/np.sum(df.new_weight)
df['cumulative_weight']=np.cumsum(df.normalized_weight)


random = np.random.rand(5)
new_lst = []
selected_indices = []  
for i in range(len(random)):
    for j in range(len(df.cumulative_weight)):
        if random[i] < df.cumulative_weight[j]:
            selected_indices.append(j)
            break  
    if len(selected_indices) == 5:
        break  
for i in selected_indices:
    selected_row = df.iloc[i:i+1, :]
    df_new = pd.concat([df_new, selected_row], ignore_index=True)

df_new.reset_index(drop=True, inplace=True)


df_new.drop(columns={"predicted_y","y*yhat","new_weight","normalized_weight","cumulative_weight"},inplace=True)
X=df_new[["Age","Salary"]]
y=df_new['Atteded_College']

model_2=DecisionTreeClassifier()
model_2.fit(X,y)
y_pre=model_2.predict(X)

df_new["predicted_y"]=y_pre
df_new["y*yhat"]=df_new.Atteded_College*df_new.predicted_y

accuracy = np.sum(y == y_pre)/len(y)
error = 1 - accuracy
alpha = 0.5*np.log((1 - error)/error)

df_new['new_weight']=df_new.weights*np.exp(-1*df_new['y*yhat']*alpha)

df_new['normalized_weight']=(df_new['new_weight']/np.sum(df_new['new_weight']))

df_new['cumulative_weight']=np.cumsum(df_new.normalized_weight)